In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#2. specify parameters
pipeline_params={
}
step_params={
}
substep_params={
    "FILTER_EMPTY_GT"    : False,
    "AUGMENTATION_TYPE"  : 1,
    "MIN_OBJECT_SIZE"    : 5,
    "MAX_SIZE"           : 1024,
    "KEEP_RATIO"         : True
}

In [3]:
#3 define substep interface
from sinara.substep import NotebookSubstep, default_param_values, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
    outputs = 
    [
        { ENTITY_NAME: "train_coco_data"},
        { ENTITY_NAME: "eval_coco_data"},
        { ENTITY_NAME: "cache_config"}
    ],
    tmp_outputs =
    [
        { ENTITY_NAME: "cache_data" }
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

**OUTPUTS:**


[{'user.yolox_mmdet.zone.1_data_import.train_coco_data': '/data/home/jovyan/yolox_mmdet/zone/1_data_import/run-23-10-10-064744/train_coco_data'},
 {'user.yolox_mmdet.zone.1_data_import.eval_coco_data': '/data/home/jovyan/yolox_mmdet/zone/1_data_import/run-23-10-10-064744/eval_coco_data'},
 {'user.yolox_mmdet.zone.1_data_import.cache_config': '/data/home/jovyan/yolox_mmdet/zone/1_data_import/run-23-10-10-064744/cache_config'}]




**TMP OUTPUTS:**


[{'cache:user.yolox_mmdet.zone.1_data_import.cache_data': '/data/cache/user/yolox_mmdet/zone/1_data_import/run-23-10-10-064744/cache_data'}]




In [4]:
#4 write outputs
outputs = substep.outputs()
tmp_outputs = substep.tmp_outputs()

In [5]:
#5 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

Session is run


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/10 06:47:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'http://localhost:4040'

#### Dataset description:

The COCO (Common Objects in Context) dataset is a large-scale image recognition dataset for object detection, segmentation, and captioning tasks. It contains over 330,000 images, each annotated with 80 object categories and 5 captions describing the scene. The COCO dataset is widely used in computer vision research and has been used to train and evaluate many state-of-the-art object detection and segmentation models.

The dataset has two main parts: the images and their annotations.

The images are organized into a hierarchy of directories, with the top-level directory containing subdirectories for the train, validation, and test sets.
The annotations are provided in JSON format, with each file corresponding to a single image.
Each annotation in the dataset includes the following information:

Image file name
Image size (width and height)
List of objects with the following information: Object class (e.g., "person," "car"); Bounding box coordinates (x, y, width, height); Segmentation mask (polygon or RLE format); Keypoints and their positions (if available)
Five captions describing the scene
The COCO dataset also provides additional information, such as image super categories, license, and coco-stuff (pixel-wise annotations for stuff classes in addition to 80 object classes).

MS COCO offers various types of annotations,

Object detection with bounding box coordinates and full segmentation masks for 80 different objects
Stuff image segmentation with pixel maps displaying 91 amorphous background areas
Panoptic segmentation identifies items in images based on 80 "things" and 91 "stuff" categories
Dense pose with over 39,000 photos featuring over 56,000 tagged persons with a mapping between pixels and a template 3D model and natural language descriptions for each image
Keypoint annotations for over 250,000 persons annotated with key points such as the right eye, nose, and left hip

Dataset load process    
Download dataset from the internet storage      

In [13]:
#5 load dataset 
data_url = "http://images.cocodataset.org/zips/val2017.zip"
annot_url = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"

In [14]:
outputs

OutputUrls(full_train_coco_data='user.yolox_mmdet.zone.1_data_import.train_coco_data', train_coco_data='/data/home/jovyan/yolox_mmdet/zone/1_data_import/run-23-10-10-064744/train_coco_data', full_eval_coco_data='user.yolox_mmdet.zone.1_data_import.eval_coco_data', eval_coco_data='/data/home/jovyan/yolox_mmdet/zone/1_data_import/run-23-10-10-064744/eval_coco_data', full_cache_config='user.yolox_mmdet.zone.1_data_import.cache_config', cache_config='/data/home/jovyan/yolox_mmdet/zone/1_data_import/run-23-10-10-064744/cache_config')

In [15]:
print(f"{outputs.train_coco_data=}")
print(f"{outputs.eval_coco_data=}")
print(f"{outputs.cache_config=}")

outputs.train_coco_data='/data/home/jovyan/yolox_mmdet/zone/1_data_import/run-23-10-10-064744/train_coco_data'
outputs.eval_coco_data='/data/home/jovyan/yolox_mmdet/zone/1_data_import/run-23-10-10-064744/eval_coco_data'
outputs.cache_config='/data/home/jovyan/yolox_mmdet/zone/1_data_import/run-23-10-10-064744/cache_config'


In [16]:
print(f"{tmp_outputs.cache_data=}")

tmp_outputs.cache_data='/data/cache/user/yolox_mmdet/zone/1_data_import/run-23-10-10-064744/cache_data'


In [17]:
import os.path as osp
import os

In [18]:
os.makedirs(tmp_outputs.cache_data)

In [19]:
!wget {data_url} -O {osp.join(tmp_outputs.cache_data, osp.basename(data_url))}
!wget {annot_url} -O {osp.join(tmp_outputs.cache_data, osp.basename(annot_url))}

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/jovyan/.wget-hsts'. HSTS will be disabled.
--2023-10-10 06:49:17--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.212.217, 16.182.42.81, 52.217.254.17, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.212.217|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘/data/cache/user/yolox_mmdet/zone/1_data_import/run-23-10-10-064744/cache_data/val2017.zip’

/data/cache/user/yo 100%[===================>] 777.80M  8.53MB/s    in 1m 42s  

2023-10-10 06:50:59 (7.66 MB/s) - ‘/data/cache/user/yolox_mmdet/zone/1_data_import/run-23-10-10-064744/cache_data/val2017.zip’ saved [815585330/815585330]

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store 

In [20]:
#6 unzip dataset
!unzip -q {osp.join(tmp_outputs.cache_data, osp.basename(data_url))} -d {tmp_outputs.cache_data}
!unzip -q {osp.join(tmp_outputs.cache_data, osp.basename(annot_url))} -d {tmp_outputs.cache_data}

In [22]:
!ls {tmp_outputs.cache_data}

annotations  annotations_trainval2017.zip  val2017  val2017.zip


In [25]:
from utils.coco import load as load_coco

ImportError: libcudart.so.10.2: cannot open shared object file: No such file or directory

In [24]:
import mmcv

In [16]:
#7 save dataset to parquet
from pathlib import Path
from tqdm import tqdm
import pandas as pd


def get_files(path, extensions, relative=True):

    if type(extensions) is str:
        extensions = [extensions]

    all_files = []
    for ext in extensions:
        if relative:
            all_files.extend(Path(path).rglob(ext))
        else:
            all_files.extend(Path(path).glob(ext))

    for i in range(len(all_files)):
        all_files[i] = str(all_files[i])

    return all_files

def build_pandas_df(_dir):
    pack = []
    files = get_files(_dir, "*")
    for file in tqdm(files):        
        row = {
            "file_names"       : None,
            "files_binary"     : None,
        }
        
        with open(file, 'rb') as file_fd:
            row['files_binary'] = file_fd.read()
        
        row["file_names"] = osp.basename(file)        
        pack.append(row.copy())
    
    return pack

/opt/conda/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [17]:
#8 save image and annotations to parquets 
data_df = build_pandas_df(osp.join(outputs.cache_data, "val2017"))
data_df = spark.createDataFrame(data_df)
data_df.printSchema()

annot_df = build_pandas_df(osp.join(outputs.cache_data, "annotations"))
annot_df = spark.createDataFrame(annot_df)
annot_df.printSchema()

print(f"{outputs.coco_val_img=}")
print(f"{outputs.coco_val_annot=}")

100%|██████████| 5000/5000 [00:38<00:00, 128.21it/s]


root
 |-- file_names: string (nullable = true)
 |-- files_binary: binary (nullable = true)



100%|██████████| 6/6 [00:01<00:00,  5.23it/s]


root
 |-- file_names: string (nullable = true)
 |-- files_binary: binary (nullable = true)

outputs.coco_val_img='/data/home/jovyan/yolox_mmdet/zone/1_data_import/run-23-10-10-000409/coco_val_img'
outputs.coco_val_annot='/data/home/jovyan/yolox_mmdet/zone/1_data_import/run-23-10-10-000409/coco_val_annot'


In [ ]:
# data_df.write.mode("overwrite").option("compression", "none").parquet(outputs.coco_val_img)
# annot_df.write.mode("overwrite").option("compression", "none").parquet(outputs.coco_val_annot)

In [18]:
#9 stop spark
SinaraSpark.stop_session()